<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Meta/blob/master/Demo_China_A_share_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quantitative trading in China A stock market with FinRL

In [1]:
import os
import time
import sys
from copy import deepcopy

import torch
import torch.nn as nn
import numpy as np
import numpy.random as rd
import pandas as pd

### Download training data
Download training data (2.1MB) from github: `https://github.com/Yonv1943/Python/blob/master/scow`
And save in current working path :`.`

训练数据是中国A股 1113 天的股票价格，以及其他与交易相关的因子。

In [3]:
!wget https://github.com/Yonv1943/Python/blob/master/scow/China_A_shares.pandas.dataframe

--2022-04-02 16:30:34--  https://github.com/Yonv1943/Python/blob/master/scow/China_A_shares.pandas.dataframe
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘China_A_shares.pandas.dataframe.3’

China_A_shares.pand     [ <=>                ] 120.73K  --.-KB/s    in 0.1s    

2022-04-02 16:30:35 (1.12 MB/s) - ‘China_A_shares.pandas.dataframe.3’ saved [123624]



### 金融仿真环境部分 FinRL
相关代码来自[FinRL-Meta Demo_China_A_share_market](https://github.com/AI4Finance-Foundation/FinRL-Meta/blob/master/Demo_China_A_share_market.ipynb)

我们将这1113天的中国A股数据，简单地分成：
- 训练集占75%，0~834天
- 测试集站25% 834~1113天
分钟级甚至秒级的数据的数据量将会很多，我们这里只是一个简单的教程，因为数据太少，所以我们没有划分验证集（validation），请读者注意。

这个的交易训练环境 env（或者说仿真器 simulator）的超参数配置如下：

```
class StockTradingEnv:
    def __init__(self, 
        initial_amount=1e6,  # 初始本金
        max_stock=1e2,  # 最大交易额度，买入或卖出100个单位
        buy_cost_pct=1e-3,  # 交易损耗率设为 0.001
        sell_cost_pct=1e-3,  # 交易损耗率设为 0.001
        gamma=0.99,  # 强化学习的折扣比率，给人为设置的终止状态的reward进行补偿的时候会用到
        beg_idx=0,   # 使用数据集的这个位置之后的数据
        end_idx=1113  # 使用数据集的这个位置之前的数据
     ):
```

cumulative_returns 表示从开始到结束智能体交易获得的收益。为了方便，我们直接显示了「本金的增长倍数」。
如 1.19 表示在一段时间的交易后，本金增长了1.19倍。
- random action 表示交易动作是随机的 `action=rd.uniform(-1, 1, action_dim)`，强化学习训练得到的智能体不应该比这个分数低。
- buy all share 表示一直使用最大的额度买入所有股票 `action=np.ones(action_dim)`，能一定程度上反映大盘走势

我们为了方便演示和学习，选择了较少的数据，从而把训练时间压缩到1000秒内。实际上，训练数据需要再多一个数量级。

In [5]:
from demo_FinRL_ElegantRL_China_A_shares import StockTradingEnv
from demo_FinRL_ElegantRL_China_A_shares import check_env

check_env()

| StockTradingEnv: close_ary.shape (279, 15)
| StockTradingEnv: tech_ary.shape (279, 120)

cumulative_returns of random action:      1.24
cumulative_returns of random action:      1.28
cumulative_returns of random action:      1.53
cumulative_returns of random action:      1.29

cumulative_returns of buy all share:      1.19
cumulative_returns of buy all share:      1.19
cumulative_returns of buy all share:      1.19
cumulative_returns of buy all share:      1.19



### 深度强化学习部分 ElegantRL

相关代码来自[ElegantRL helloworld](https://github.com/AI4Finance-Foundation/ElegantRL/tree/master/elegantrl_helloworld)

我们使用ElegantRL库的代码，连接上FinRL的仿真环境，加载中国A股的数据，调用`run()`函数，训练一个自动交易智能体。

`run()` 函数将会用到以下内容：

```
# env.py
class StockTradingEnv()  # 来自FinRL库的交易仿真环境
def build_env()  # 创建训练仿真环境
def get_gym_env_args()  # 获得仿真环境的参数

# net.py
class ActorPPO  # PPO算法的策略网络
class CriticPPO  # PPO算法的价值网络

# agent.py
class AgentPPO  # PPO算法的主体
class ReplayBufferList  # 经验回放缓存（存放强化学习的训练数据）

# run.py
class Arguments  # 强化学习的超参数（可以看这个类的注释了解超参数的作用）
def train_agent()  # 训练强化学习智能体的函数
```

**记得选则你训练用的GPU卡ID**，填上0表示用ID编号为0 的GPU卡`run(gpu_id=0)`。如果没有GPU，可以填`-1`，或者不做任何处理，代码会自己选择。

我们用的数据比较少，用笔记本也能在500秒内完成训练，想要缩短训练时间，可以把训练步数改小`args.break_step = int(5e5)`

### Import modules

In [8]:
from demo_FinRL_ElegantRL_China_A_shares import StockTradingEnv
from demo_FinRL_ElegantRL_China_A_shares import get_gym_env_args
from demo_FinRL_ElegantRL_China_A_shares import AgentPPO
from demo_FinRL_ElegantRL_China_A_shares import Arguments
from demo_FinRL_ElegantRL_China_A_shares import train_agent

def run(gpu_id=-1):
    env = StockTradingEnv()
    env_func = StockTradingEnv
    env_args = get_gym_env_args(env=env, if_print=False)
    env_args['beg_idx'] = 0  # training set
    env_args['end_idx'] = 834  # training set

    args = Arguments(AgentPPO, env_func=env_func, env_args=env_args)
    args.target_step = args.max_step * 4
    args.reward_scale = 2 ** -7
    args.learning_rate = 2 ** -14
    args.break_step = int(5e5)

    args.learner_gpus = gpu_id
    args.random_seed += gpu_id + 1943
    train_agent(args)

run()

| StockTradingEnv: close_ary.shape (1113, 15)
| StockTradingEnv: tech_ary.shape (1113, 120)
| Arguments Remove cwd: ./StockTradingEnv-v2_PPO_-1
| StockTradingEnv: close_ary.shape (834, 15)
| StockTradingEnv: tech_ary.shape (834, 120)
Step:5.00e+03  ExpR:    0.08  Returns:    1.55  ObjC:    3.70  ObjA:    0.05  
Step:6.00e+04  ExpR:    0.32  Returns:    3.04  ObjC:    3.16  ObjA:   -0.02  
Step:1.15e+05  ExpR:    0.40  Returns:    3.27  ObjC:    1.67  ObjA:   -0.05  
Step:1.70e+05  ExpR:    0.42  Returns:    4.00  ObjC:    1.31  ObjA:    0.17  
Step:2.25e+05  ExpR:    0.50  Returns:    4.12  ObjC:    0.97  ObjA:    0.16  
Step:2.80e+05  ExpR:    0.58  Returns:    4.10  ObjC:    0.99  ObjA:    0.01  
Step:3.35e+05  ExpR:    0.62  Returns:    4.66  ObjC:    1.25  ObjA:    0.03  
Step:3.90e+05  ExpR:    0.67  Returns:    4.74  ObjC:    1.02  ObjA:    0.20  
Step:4.45e+05  ExpR:    0.73  Returns:    5.90  ObjC:    0.90  ObjA:   -0.13  
Step:5.00e+05  ExpR:    0.77  Returns:    5.50  ObjC:  

In [2]:
训练结束后，会有很多策略网络的模型文件保存在 `| Arguments Remove cwd: ./StockTradingEnv-v2_PPO_-1`
    
你也可以在训练前，修改这个超参数，自己选择保存目录位置`args.cwd=./current_working_directory`

我接下来使用函数`evaluate_models_in_directory(dir_path='./StockTradingEnv-v2_PPO_-1')`评估保存在目录中的模型文件。

In [13]:
from demo_FinRL_ElegantRL_China_A_shares import evaluate_models_in_directory

evaluate_models_in_directory(dir_path='./StockTradingEnv-v2_PPO_-1')

| evaluate_models_in_directory: gpu_id -1
| evaluate_models_in_directory: dir_path ./StockTradingEnv-v2_PPO_-1
| StockTradingEnv: close_ary.shape (279, 15)
| StockTradingEnv: tech_ary.shape (279, 120)
cumulative_returns     1.093  actor_00000000004998_00000003_00000.08.pth
cumulative_returns     1.067  actor_00000000059976_00000033_00000.32.pth
cumulative_returns     1.107  actor_00000000114954_00000062_00000.40.pth
cumulative_returns     1.090  actor_00000000169932_00000093_00000.42.pth
cumulative_returns     1.080  actor_00000000224910_00000122_00000.50.pth
cumulative_returns     1.147  actor_00000000279888_00000152_00000.58.pth
cumulative_returns     1.257  actor_00000000334866_00000183_00000.62.pth
cumulative_returns     1.295  actor_00000000389844_00000213_00000.67.pth
cumulative_returns     1.316  actor_00000000444822_00000243_00000.73.pth
cumulative_returns     1.299  actor_00000000499800_00000273_00000.77.pth


cumulative_returns 表示从开始到结束智能体交易获得的收益。为了方便，我们直接显示了「本金的增长倍数」。

`actor_00000000004998_00000003_00000.08.pth` 是训练时保存的策略网络模型文件 `actor`，表示它是在环境中采样4998步，训练3秒，探索环境的得分是0.08的一个模型，它的实际得分是 1.093，表示在一段时间的交易后，本金增长了1.093倍。

欢迎使用 [FinRL-Meta Demo_China_A_share_market](https://github.com/AI4Finance-Foundation/FinRL-Meta/blob/master/Demo_China_A_share_market.ipynb) 以及 [ElegantRL helloworld](https://github.com/AI4Finance-Foundation/ElegantRL/tree/master/elegantrl_helloworld) 的开源代码做出比这个例子更好的东西。

如果想要交流更多与金融强化学习相关的话题，我们推荐：
- 英文社区 [FinRL的 Slack channel](https://join.slack.com/t/ai4financeworkspace/shared_invite/zt-v670l1jm-dzTgIT9fHZIjjrqprrY0kg)
- 中文社区 [深度强化学习实验室网站：FinR L板块](http://deeprlhub.com/t/finrl)。这是讨论深度强化学习的网站，有FinRL板块。与QQ群不同的，讨论的记录，很方便查看。
- QQ群 “金融强化学习 FinRL”（群号341070204），它是 “深度强化学习 ElegantRL”的姊妹群，原来的群长期处于满2000人的状态，所以我在2022-03-31日创建了这个新群。新群有了解金融强化学习的多位成员，一起讨论，一起成长。 

### Authors
github username YonV1943，知乎 曾伊言